<a href="https://colab.research.google.com/github/pedrohp09/Machine-Learning/blob/main/Exerc%C3%ADcio_8_AutoML_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aluno: Pedro Henrique Pereira do Nascimento Lista de Exercícios da disciplina de Machine Learning Exercício 8

AutoML: Hyperparameter Tuning

Realize a otimização de hiperparâmetros utilizando uma ferramenta de AutoML em um modelo de árvore de decisão ou floresta aleatória. Avalie o desempenho do modelo otimizado.




In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier


from tpot import TPOTClassifier

import warnings
warnings.filterwarnings('ignore')


In [3]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target
df['species'] = df['target'].apply(lambda x: iris.target_names[x])


print("Primeiras linhas do conjunto de dados:")
print(df.head())

print("\nInformações do conjunto de dados:")
print(df.info())

print("\nValores faltantes por coluna:")
print(df.isnull().sum())


Primeiras linhas do conjunto de dados:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target species  
0       0  setosa  
1       0  setosa  
2       0  setosa  
3       0  setosa  
4       0  setosa  

Informações do conjunto de dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-nul

In [4]:
X = df.drop(['target', 'species'], axis=1)
y = df['target']


In [5]:
# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Tamanho do conjunto de treino: {X_train.shape[0]} amostras')
print(f'Tamanho do conjunto de teste: {X_test.shape[0]} amostras')


Tamanho do conjunto de treino: 120 amostras
Tamanho do conjunto de teste: 30 amostras


In [6]:
# Inicializando o modelo com hiperparâmetros padrão
rf = RandomForestClassifier(random_state=42)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Precisão do Random Forest (Hiperparâmetros Padrão): {accuracy_rf * 100:.2f}%')

print("\nRelatório de Classificação (Random Forest):")
print(classification_report(y_test, y_pred_rf, target_names=iris.target_names))


Precisão do Random Forest (Hiperparâmetros Padrão): 90.00%

Relatório de Classificação (Random Forest):
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.82      0.90      0.86        10
   virginica       0.89      0.80      0.84        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30



In [7]:
tpot = TPOTClassifier(
    generations=5,
    population_size=50,
    verbosity=2,
    random_state=42,
    config_dict='TPOT sparse'  # Para permitir apenas classificadores com parâmetros padrão
)

tpot.fit(X_train, y_train)

y_pred_tpot = tpot.predict(X_test)

accuracy_tpot = accuracy_score(y_test, y_pred_tpot)
print(f'\nPrecisão do TPOT (AutoML): {accuracy_tpot * 100:.2f}%')

print("\nRelatório de Classificação (TPOT):")
print(classification_report(y_test, y_pred_tpot, target_names=iris.target_names))


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.975

Generation 2 - Current best internal CV score: 0.975

Generation 3 - Current best internal CV score: 0.975

Generation 4 - Current best internal CV score: 0.975

Generation 5 - Current best internal CV score: 0.975

Best pipeline: KNeighborsClassifier(input_matrix, n_neighbors=43, p=2, weights=distance)

Precisão do TPOT (AutoML): 93.33%

Relatório de Classificação (TPOT):
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.90      0.90      0.90        10
   virginica       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

